# Analyze NaN in GFA Summary Files

In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.table import Table
gfa_dir = os.path.join(os.environ['DESI_ROOT'], 'survey', 'GFA')
sv1_gfa_file = os.path.join(gfa_dir, 'offline_matched_coadd_ccds_SV1-thru_20210928.fits')
main_gfa_file = os.path.join(gfa_dir, 'offline_matched_coadd_ccds_main-thru_20240828.fits')

In [2]:
with fits.open(sv1_gfa_file, mode='readonly') as hdulist:
    sv1_gfa_data = hdulist[2].data
with fits.open(main_gfa_file, mode='readonly') as hdulist:
    main_gfa_data = hdulist[2].data

In [3]:
gfa_nan_summary = dict()
# EXPID, NIGHT, In SV1, In main, TRANSPARENCY,
columns_from_gfa = ("TRANSPARENCY", "FWHM_ASEC", "FIBER_FRACFLUX", "FIBER_FRACFLUX_ELG",
                    "FIBER_FRACFLUX_BGS", "FIBERFAC", "FIBERFAC_ELG", "FIBERFAC_BGS",
                    "SKY_MAG_AB", "AIRMASS")
for column in columns_from_gfa:
    w_sv1 = np.where(~np.isfinite(sv1_gfa_data[column]))[0]
    if len(w_sv1) > 0:
        # print('{0} NaN found in column {1} for SV1.'.format(len(w_sv1), column))
        sv1_expid = sv1_gfa_data['EXPID'][w_sv1].tolist()
        sv1_night = sv1_gfa_data['NIGHT'][w_sv1].tolist()
        for i, expid in enumerate(sv1_expid):
            if expid in gfa_nan_summary:
                assert sv1_night[i] == gfa_nan_summary[expid]['night']
                gfa_nan_summary[expid]['sv1_nan'].append(column)
            else:
                gfa_nan_summary[expid] = {'night': sv1_night[i],
                                          'sv1_nan': [column],
                                          'main_nan': []}

    w_main = np.where(~np.isfinite(main_gfa_data[column]))[0]
    if len(w_main) > 0:
        # print('{0} NaN found in column {1} for main.'.format(len(w_main), column))
        main_expid = main_gfa_data['EXPID'][w_main].tolist()
        main_night = main_gfa_data['NIGHT'][w_main].tolist()
        for i, expid in enumerate(main_expid):
            if expid in gfa_nan_summary:
                assert main_night[i] == gfa_nan_summary[expid]['night']
                gfa_nan_summary[expid]['main_nan'].append(column)
            else:
                gfa_nan_summary[expid] = {'night': main_night[i],
                                          'sv1_nan': [],
                                          'main_nan': [column]}

In [4]:
# print(gfa_nan_summary)
summary_columns = ['EXPID', 'NIGHT'] + sorted(columns_from_gfa)
summary_table = dict([(c, list()) for c in summary_columns])
for expid in sorted(gfa_nan_summary.keys()):
    summary_table['EXPID'].append(expid)
    summary_table['NIGHT'].append(gfa_nan_summary[expid]['night'])
    if gfa_nan_summary[expid]['sv1_nan'] and gfa_nan_summary[expid]['main_nan']:
        assert len(gfa_nan_summary[expid]['sv1_nan']) == len(gfa_nan_summary[expid]['main_nan'])
        assert tuple(sorted(gfa_nan_summary[expid]['sv1_nan'])) == tuple(sorted(gfa_nan_summary[expid]['main_nan']))
        entries = 2
    else:
        entries = 1
    for column in columns_from_gfa:
        if column in gfa_nan_summary[expid]['sv1_nan'] or column in gfa_nan_summary[expid]['main_nan']:
            summary_table[column].append(entries)
        else:
            summary_table[column].append(0)
# print(summary_table)

In [5]:
t = Table()
for c in summary_columns:
    t[c] = np.array(summary_table[c])
t.write(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'gfa_nan_summary.csv'), format='ascii.csv')